# Minneapolis Neighborhood Demographic and Crime Data

## Overview

#### This notebook contains the python code to populate a database that contains tables: 1. neighborhood; 2. crime_data; 3. demo_cat; 4. demographic_data.  The data for these tables comes from csv files containing the crime data for minneapolis neighborhoods and demographic (age, income, education) data for each neighborhood.

* The crime data comes from one csv file (Crime_Data.csv) found at:  https://www.minneapolismn.gov/government/government-data/datasource/crime-dashboard/

* The demographic data for each neighborhood (individual files for each demographic/neighborhood combination) found at: https://www.mncompass.org/profiles/neighborhoods/minneapolis-saint-paul

#### *Please note that you need to have created the database and tables prior to running this notebook to load the data

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, inspect
import re
import os

## Extract

* The Crime_Data.csv file is loaded into a pandas dataframe using pd.read_csv
* For each demographic (age, income, education) there is a directory with one csv file per neighborhood
    * Create a dictionary of the files in the demographic directory
    * Print the dictionary and copy the results into a new cell to create a new dictionary
    * Manually set the key to the neighborhood name (matching what is in the Crime file)
    * Create the demographic dataframe (age, income, education)
    * Loop through the dictionary
        * Load the CSV for the neighborhood into a temporary dataframe
        * Add columns to hold the neighborhood and demographic category (age, income, education)
        * Append the temporary dataframe to the demographic dataframe (age, income, education)


** special thanks to TA Colin for the help on the approach to the demographic files and their naming

In [ ]:
#Extract Crime_Data.csv into minneapolis_crime_df dataframe
minneapolis_crime_csv = "Resources/Crime_Data.csv"
minneapolis_crime_df = pd.read_csv(minneapolis_crime_csv)

In [ ]:
#### ----  Extract Age Demographic csvs ---- ####

#Create a Dictionary of the list age demographic files (one per neighborhood) where the filename is the key and value
dict_str = '{'
for file in os.listdir("./Resources/age"):
    dict_str += f"'{file}': '{file}', \n"
dict_str += '}'

#Print the dictionary
print(dict_str)

In [ ]:
#Created a new dictionary by copying the text output from above into this cell and manually updating the key 
#field in the dictionary to be the neighborhood name - making sure to match the name from the Crime_Data.csv

age = {'Windom': 'windom-neighborhood-popu.csv', 
'Downtown East': 'downtown-east-pop.csv', 
'Hawthorne': 'hawthorne-neighborhood-p.csv', 
'Midtown Phillips': 'midtown-phillips-neighbo.csv', 
'Bancroft': 'bancroft-pop.csv', 
'Downtown West': 'downtown-west-pop.csv', 
'Marcy Holmes': 'marcy-holmes-neighborhoo.csv', 
'Near - North': 'near-north-neighborhood.csv', 
'Audubon Park': 'audubon-park-pop.csv', 
'East Bde Maka Ska': 'east-bde-maka-ska-pop.csv', 
'Jordan': 'jordan-neighborhood-popu.csv', 
'Powderhorn Park': 'powderhorn-park-neighbor.csv', 
'South Uptown': 'south-uptown-neighborhoo.csv', 
'Cleveland': 'cleveland-pop.csv', 
'Bottineau': 'bottineau-pop.csv', 
'Standish': 'standish-neighborhood-po.csv', 
'Victory': 'victory-neighborhood-pop.csv', 
'North Loop': 'north-loop-neighborhood.csv', 
'University of Minnesota': 'university-of-minnesota.csv', 
'Columbia Park': 'columbia-park-pop.csv', 
'Cedar - Isles - Dean': 'cedar-isles-dean-pop.csv', 
'Webber - Camden': 'webber-camden-neighborho.csv', 
'Wenonah': 'wenonah-neighborhood-pop.csv', 
'Northrop': 'northrop-neighborhood-po.csv', 
'Cedar Riverside': 'cedar-riverside-pop.csv', 
'Phillips West': 'phillips-west-neighborho.csv', 
'Sumner - Glenwood': 'sumner-glenwood-neighbor.csv', 
'Armatage': 'armatage-pop.csv', 
'Bryant': 'bryant-pop.csv', 
'Shingle Creek': 'shingle-creek-neighborho.csv', 
'Northeast Park': 'northeast-park-neighborh.csv', 
'Lind - Bohanon': 'lind-bohanon-neighborhoo.csv', 
'Lowry Hill': 'lowry-hill-neighborhood.csv', 
'Elliot Park': 'elliot-park-neighborhood.csv', 
'Minnehaha': 'minnehaha-neighborhood-p.csv', 
'Bryn - Mawr': 'bryn-mawr-pop.csv', 
'Holland': 'holland-neighborhood-pop.csv', 
'Longfellow': 'longfellow-neighborhood.csv', 
'Lyndale': 'lyndale-neighborhood-pop.csv', 
'East Isles': 'east-isles-neighborhood.csv', 
'Como': 'como-pop.csv', 
'Kenwood': 'kenwood-neighborhood-pop.csv', 
'Tangletown': 'tangletown-neighborhood.csv', 
'East Harriet': 'east-harriet-pop.csv', 
'Linden Hills': 'linden-hills-neighborhoo.csv', 
'Loring Park': 'loring-park-neighborhood.csv', 
'Lowry Hill East': 'lowry-hill-east-neighbor.csv', 
'Prospect Park - East River Road': 'prospect-park-east-river.csv', 
'Folwell': 'folwell-neighborhood-pop.csv', 
'McKinley': 'mckinley-neighborhood-po.csv', 
'Fulton': 'fulton-neighborhood-popu.csv', 
'Hale': 'hale-neighborhood-popula.csv', 
'Ventura Village': 'ventura-village-neighbor.csv', 
'West Maka Ska': 'west-maka-ska-neighborho.csv', 
'Harrison': 'harrison-neighborhood-po.csv', 
'King Field': 'king-field-neighborhood.csv', 
'St. Anthony East': 'st-anthony-east-neighbor.csv', 
'Page': 'page-neighborhood-popula.csv', 
'Morris Park': 'morris-park-neighborhood.csv', 
'Central': 'central-pop.csv', 
'Keewaydin': 'keewaydin-neighborhood-p.csv', 
'Beltrami': 'beltrami-pop.csv', 
'Steven\'s Square - Loring Heights': 'stevens-square-loring-he.csv', 
'Willard - Hay': 'willard-hay-neighborhood.csv', 
'Hiawatha': 'hiawatha-neighborhood-po.csv', 
'Seward': 'seward-neighborhood-popu.csv', 
'Ericsson': 'ericsson-neighborhood-po.csv', 
'St. Anthony West': 'st-anthony-west-neighbor.csv', 
'Marshall Terrace': 'marshall-terrace-neighbo.csv', 
'Diamond Lake': 'diamond-lake-pop.csv', 
'East Phillips': 'east-phillips-neighborho.csv', 
'Sheridan': 'sheridan-neighborhood-po.csv', 
'Windom Park': 'windom-park-neighborhood.csv', 
'Howe': 'howe-neighborhood-popula.csv', 
'Mid - City Industrial': 'mid-city-industrial-neig.csv', 
'Lynnhurst': 'lynnhurst-neighborhood-p.csv', 
'Logan Park': 'logan-park-neighborhood.csv', 
'Nicollet Island - East Bank': 'nicollet-island-east-ban.csv', 
'Kenny': 'kenny-neighborhood-popul.csv', 
'Whittier': 'whittier-neighborhood-po.csv', 
'Waite Park': 'waite-park-neighborhood.csv', 
'Regina': 'regina-neighborhood-popu.csv', 
'Corcoran': 'corcoran-neighborhood-po.csv', 
'Field': 'field-neighborhood-popul.csv', 
'Cooper': 'cooper-neighborhood-popu.csv', 
'all': 'minneapolis-population-b.csv' 
}


#Create the age demographics dataframe
age_df = pd.DataFrame()

#Loop through the dictionary of neighborhood files
for key in age:
    #Load the current neighborhood csv to temporary dataframe
    age_csv = f"./Resources/age/{age[key]}"
    temp_age_df = pd.read_csv(age_csv)
    
    #Add the neighborhood column to the dataframe and populate with the neighborhood name from the dictionary key
    temp_age_df['neighborhood'] = key
    temp_age_df['demographic'] = 'age'
    
    #Append the neighborhood dataframe to the age dataframe
    age_df = pd.concat([age_df, temp_age_df], ignore_index = True)

age_df

In [ ]:
#### ----  Extract Income Demographic csvs ---- ####

#Create a Dictionary of the list age demographic files (one per neighborhood) where the filename is the key and value
dict_str = '{'
for file in os.listdir("./Resources/income"):
    dict_str += f"'{file}': '{file}', \n"
dict_str += '}'

#Print the dictionary
print(dict_str)

In [ ]:
#Created a new dictionary by copying the text output from above into this cell and manually updating the key 
#field in the dictionary to be the neighborhood name - making sure to match the name from the Crime_Data.csv
income =   {'Bottineau': 'bottineau-income.csv',
'Holland': 'holland-neighborhood-hou.csv',
'Bryn - Mawr': 'bryn-mawr-income.csv',
'Lyndale': 'lyndale-neighborhood-hou.csv',
'Beltrami': 'beltrami-income.csv',
'Kenwood': 'kenwood-neighborhood-hou.csv',
'Midtown Phillips': 'midtown-phillips-neighbo.csv',
'Lynnhurst': 'lynnhurst-neighborhood-h.csv',
'Howe': 'howe-neighborhood-househ.csv',
'Standish': 'standish-neighborhood-ho.csv',
'Como': 'como-income.csv',
'Armatage': 'armatage-income.csv',
'Marcy Holmes': 'marcy-holmes-neighborhoo.csv',
'Near - North': 'near-north-neighborhood.csv',
'Powderhorn Park': 'powderhorn-park-neighbor.csv',
'Downtown East': 'downtown-east-income.csv',
'South Uptown': 'south-uptown-neighborhoo.csv',
'East Bde Maka Ska': 'east-bde-maka-ska-income.csv',
'Wenonah': 'wenonah-neighborhood-hou.csv',
'Fulton': 'fulton-neighborhood-hous.csv',
'Central': 'central-income.csv',
'North Loop': 'north-loop-neighborhood.csv',
'University of Minnesota': 'university-of-minnesota.csv',
'Victory': 'victory-neighborhood-hou.csv',
'Bryant': 'bryant-income.csv',
'Seward': 'seward-neighborhood-hous (1).csv',
'Page': 'page-neighborhood-househ.csv',
'Webber - Camden': 'webber-camden-neighborho.csv',
'Cedar - Isles - Dean': 'cedar-isles-dean-income.csv',
'Phillips West': 'phillips-west-neighborho.csv',
'Hale': 'hale-neighborhood-househ.csv',
'Sumner - Glenwood': 'sumner-glenwood-neighbor.csv',
'Keewaydin': 'keewaydin-neighborhood-h.csv',
'Shingle Creek': 'shingle-creek-neighborho.csv',
'Northeast Park': 'northeast-park-neighborh.csv',
'Lind - Bohanon': 'lind-bohanon-neighborhoo.csv',
'Cooper': 'cooper-neighborhood-hous.csv',
'Lowry Hill': 'lowry-hill-neighborhood.csv',
'Northrop': 'northrop-neighborhood-ho.csv',
'Elliot Park': 'elliot-park-neighborhood.csv',
'Regina': 'regina-neighborhood-hous.csv',
'Cleveland': 'cleveland-income.csv',
'Longfellow': 'longfellow-neighborhood.csv',
'East Isles': 'east-isles-neighborhood.csv',
'Tangletown': 'tangletown-neighborhood.csv',
'Linden Hills': 'linden-hills-neighborhoo.csv',
'Loring Park': 'loring-park-neighborhood.csv',
'Lowry Hill East': 'lowry-hill-east-neighbor.csv',
'Harrison': 'harrison-neighborhood-ho.csv',
'Windom': 'windom-neighborhood-hous.csv',
'Prospect Park - East River Road': 'prospect-park-east-river.csv',
'East Harriet': 'east-harriet-income.csv',
'Minnehaha': 'minnehaha-neighborhood-h.csv',
'Cedar Riverside': 'cedar-riverside-income.csv',
'Ventura Village': 'ventura-village-neighbor.csv',
'West Maka Ska': 'west-maka-ska-neighborho.csv',
'Jordan': 'jordan-neighborhood-hous.csv',
'King Field': 'king-field-neighborhood.csv',
'St. Anthony East': 'st-anthony-east-neighbor.csv',
'Morris Park': 'morris-park-neighborhood.csv',
'Mckinley': 'mckinley-neighborhood-ho.csv',
'Steven\'s Square - Loring Heights': 'stevens-square-loring-he.csv',        
'Willard - Hay': 'willard-hay-neighborhood.csv',
'Sheridan': 'sheridan-neighborhood-ho.csv',
'Bancroft': 'bancroft-income.csv',
'Audubon Park': 'audubon-park-income.csv',
'Downtown West': 'downtown-west-income.csv',
'Kenny': 'kenny-neighborhood-house.csv',
'Corcoran': 'corcoran-neighborhood-ho.csv',
'Field': 'field-neighborhood-house.csv',
'Whittier': 'whittier-neighborhood-ho.csv',
'St. Anthony West': 'st-anthony-west-neighbor.csv',
'Marshall Terrace': 'marshall-terrace-neighbo.csv',
'East Phillips': 'east-phillips-neighborho.csv',
'Diamond Lake': 'diamond-lake-income.csv',
'Windom Park': 'windom-park-neighborhood.csv',
'Mid - City Industrial': 'mid-city-industrial-neig.csv',
'Hawthorne': 'hawthorne-neighborhood-h.csv',
'Hiawatha': 'hiawatha-neighborhood-ho.csv',
'Columbia Park': 'columbia-park-income.csv',
'Logan Park': 'logan-park-neighborhood.csv',
'Nicollet Island - East Bank': 'nicollet-island-east-ban.csv',
'Folwell': 'folwell-neighborhood-hou.csv',
'Ericsson': 'ericsson-neighborhood-ho.csv',
'Waite Park': 'waite-park-neighborhood.csv', 
'all': 'minneapolis-households-b.csv'}


#Create the income demographics dataframe
income_df = pd.DataFrame()

#Loop through the dictionary of neighborhood files
for key in income:
    #Load the current neighborhood csv to temporary dataframe
    income_csv = f"./Resources/income/{income[key]}"
    temp_income_df = pd.read_csv(income_csv)
    
    #Add the neighborhood column to the dataframe and populate with the neighborhood name from the dictionary key
    temp_income_df['neighborhood'] = key
    temp_income_df['demographic'] = 'income'
    
    #Append the neighborhood dataframe to the income dataframe
    income_df = pd.concat([income_df, temp_income_df], ignore_index = True)

income_df

In [ ]:
#### ----  Extract Education Demographic csvs ---- ####

#Create a Dictionary of the list age demographic files (one per neighborhood) where the filename is the key and value
dict_str = '{'
for file in os.listdir("./Resources/Education"):
    dict_str += f"'{file}': '{file}', \n"
dict_str += '}'

#Print the dictionary
print(dict_str)

In [ ]:
#Created a new dictionary by copying the text output from above into this cell and manually updating the key 
#field in the dictionary to be the neighborhood name - making sure to match the name from the Crime_Data.csv
education = {'Windom': 'windom-neighborhood-educ.csv',
'Downtown East': 'downtown-east-education.csv',
'Hawthorne': 'hawthorne-neighborhood-e.csv',
'Midtown Phillips': 'midtown-phillips-neighbo.csv',
'Bancroft': 'bancroft-education.csv',
'Downtown West': 'downtown-west-education.csv',
'Marcy Holmes': 'marcy-holmes-neighborhoo.csv',
'Near - North': 'near-north-neighborhood.csv',
'Audubon Park': 'audubon-park-education.csv',
'East Bde Maka Ska': 'east-bde-maka-ska-education.csv',
'Jordan': 'jordan-neighborhood-educ.csv',
'Powderhorn Park': 'powderhorn-park-neighbor.csv',
'South Uptown': 'south-uptown-neighborhoo.csv',
'Cleveland': 'cleveland-education.csv',
'Bottineau': 'bottineau-education.csv',
'Standish': 'standish-neighborhood-ed.csv',
'Victory': 'victory-neighborhood-edu.csv',
'North Loop': 'north-loop-neighborhood.csv',
'University of Minnesota': 'university-of-minnesota.csv',
'Columbia Park': 'columbia-park-education.csv',
'Cedar - Isles - Dean': 'cedar-isles-dean-education.csv',
'Webber - Camden': 'webber-camden-neighborho.csv',
'Wenonah': 'wenonah-neighborhood-edu.csv',
'Northrop': 'northrop-neighborhood-ed.csv',
'Cedar Riverside': 'cedar-riverside-education.csv',
'Phillips West': 'phillips-west-neighborho.csv',
'Sumner - Glenwood': 'sumner-glenwood-neighbor.csv',
'Armatage': 'armatage-education.csv',
'Bryant': 'bryant-education.csv',
'Shingle Creek': 'shingle-creek-neighborho.csv',
'Northeast Park': 'northeast-park-neighborh.csv',
'Lind - Bohanon': 'lind-bohanon-neighborhoo.csv',
'Lowry Hill': 'lowry-hill-neighborhood.csv',
'Elliot Park': 'elliot-park-neighborhood.csv',
'Minnehaha': 'minnehaha-neighborhood-e.csv',
'Bryn - Mawr': 'bryn-mawr-education.csv',
'Holland': 'holland-neighborhood-edu.csv',
'Longfellow': 'longfellow-neighborhood.csv',
'Lyndale': 'lyndale-neighborhood-edu.csv',
'East Isles': 'east-isles-neighborhood.csv',
'Como': 'como-education.csv',
'Kenwood': 'kenwood-neighborhood-edu.csv',
'Tangletown': 'tangletown-neighborhood.csv',
'East Harriet': 'east-harriet-education.csv',
'Linden Hills': 'linden-hills-neighborhoo.csv',
'Loring Park': 'loring-park-neighborhood.csv',
'Lowry Hill East': 'lowry-hill-east-neighbor.csv',
'Prospect Park - East River Road': 'prospect-park-east-river.csv',
'Folwell': 'folwell-neighborhood-edu.csv',
'Mckinley': 'mckinley-neighborhood-ed.csv',
'Fulton': 'fulton-neighborhood-educ.csv',
'Hale': 'hale-neighborhood-educat.csv',
'Ventura Village': 'ventura-village-neighbor.csv',
'West Maka Ska': 'west-maka-ska-neighborho.csv',
'Harrison': 'harrison-neighborhood-ed.csv',
'King Field': 'king-field-neighborhood.csv',
'St. Anthony East': 'st-anthony-east-neighbor.csv',
'Page': 'page-neighborhood-educat.csv',
'Morris Park': 'morris-park-neighborhood.csv',
'Central': 'central-education.csv',
'Keewaydin': 'keewaydin-neighborhood-e.csv',
'Beltrami': 'beltrami-education.csv',
'Willard - Hay': 'willard-hay-neighborhood.csv',
'Hiawatha': 'hiawatha-neighborhood-ed.csv',
'Seward': 'seward-neighborhood-educ.csv',
'Ericsson': 'ericsson-neighborhood-ed.csv',
'St. Anthony West': 'st-anthony-west-neighbor.csv',
'Marshall Terrace': 'marshall-terrace-neighbo.csv',
'Diamond Lake': 'diamond-lake-education.csv',
'East Phillips': 'east-phillips-neighborho.csv',
'Sheridan': 'sheridan-neighborhood-ed.csv',
'Windom Park': 'windom-park-neighborhood.csv',
'Howe': 'howe-neighborhood-educat.csv',
'Mid - City Industrial': 'mid-city-industrial-neig.csv',
'Lynnhurst': 'lynnhurst-neighborhood-e.csv',
'Logan Park': 'logan-park-neighborhood.csv',
'Nicollet Island - East Bank': 'nicollet-island-east-ban.csv',
'Kenny': 'kenny-neighborhood-educa.csv',
'Whittier': 'whittier-neighborhood-ed.csv',
'Waite Park': 'waite-park-neighborhood.csv',
'Regina': 'regina-neighborhood-educ.csv',
'Corcoran': 'corcoran-neighborhood-ed.csv',
'Field': 'field-neighborhood-educa.csv',
'Cooper': 'cooper-neighborhood-educ.csv',
'Steven\'s Square - Loring Heights': 'stevens-square-loring-he.csv', 
'all': 'minneapolis-educational.csv'
}

#Create the education demographics dataframe
education_df = pd.DataFrame()

#Loop through the dictionary of neighborhood files
for key in education:
    #Load the current neighborhood csv to temporary dataframe
    education_csv = f"./Resources/Education/{education[key]}"
    temp_education_df = pd.read_csv(education_csv)
    
    #Add the neighborhood column to the dataframe and populate with the neighborhood name from the dictionary key
    temp_education_df['neighborhood'] = key
    temp_education_df['demographic'] = 'education'
    
    #Append the neighborhood dataframe to the education dataframe
    education_df = pd.concat([education_df, temp_education_df], ignore_index = True)

education_df

## Transform

#### Transform each of the dataframes to reduce columns, replace null values, merge data and ultimately get it into the format needed for loading to the database.

### 1. Transform Crime Data
#### 1a. Create dataset to load to the crime_data table
* Part 1: Before creating neighborhood dataframe
    * Reduce columns to neighborhood, occurred_date, offense_category, offense, latitude, longitude, crime_count
    * Eliminate rows without a neighborhood value or an Occured Date value
    * Rename the column headers
    * Insert a row for the total (all)
    * Create a column to store the month/year as string for easier use in graphing
* Part 2: After creating neighborhood dataframe (1b)
    * Insert the neighborhoodid corresponding to each neighborhood
    * Remove the neighborhood column from the crime dataframe

#### 1b. Create dataset to load to the neighborhood table
* Use the crime dataframe to get the unique neighborhood names from the crime data
* Create neighborhoodid column and populate with unique id


In [ ]:
#### ---- Crime Data 1---- ####

# Reduce the columns of the Crime Data to neighborhood, occurred_date, offense_category, offense, latitude, longitude, crime_count
reduced_minne_crime = minneapolis_crime_df[["Neighborhood","Occurred_Date","Offense_Category","Offense","Latitude","Longitude","Crime_Count"]]
reduced_minne_crime = reduced_minne_crime.sort_values("Neighborhood")

#Eliminate Records with no Neighborhood value
reduced_minne_crime = reduced_minne_crime[reduced_minne_crime['Neighborhood'].notna()]

#Remove rows with null dates
reduced_minne_crime = reduced_minne_crime[reduced_minne_crime['Occurred_Date'].notna()]

#Create month/year column for easier graphing
reduced_minne_crime['year'] = pd.DatetimeIndex(reduced_minne_crime['Occurred_Date']).year
reduced_minne_crime['month'] = pd.DatetimeIndex(reduced_minne_crime['Occurred_Date']).month
reduced_minne_crime['monthyear']  = reduced_minne_crime.apply(lambda x: f'{x["month"]}/{x["year"]}', axis=1) 

# Rename the column headers
minne_crime_transformed = reduced_minne_crime.rename(columns={"Neighborhood": "neighborhood",
                                                          "Occurred_Date": "occurred_date",
                                                          "Offense_Category": "offense_cat",
                                                          "Offense": "offense",
                                                          "Latitude": "latitude",
                                                          "Longitude": "longitude",
                                                          "Crime_Count": "crime_count"})

minne_crime_transformed

In [ ]:
#### ---- Neighborhood ---- ####

#Get the list of unique neighborhoods
neighborhood = reduced_minne_crime["Neighborhood"].unique()
#Put the list into a dataframe
neighborhood_df = pd.DataFrame(neighborhood)
#Name the neighborhood column
neighborhood_df = neighborhood_df.rename(columns={0: "neighborhood"})
#Create the ID column for SQL table
neighborhood_df['neighborhoodid'] = neighborhood_df.index + 1
#Insert ID for all (total for all neighborhoods)
new_row = pd.DataFrame({'neighborhoodid':100, 'neighborhood':'all'}, index=[0])
neighborhood_df = pd.concat([new_row,neighborhood_df.loc[:]]).reset_index(drop=True)

neighborhood_df

In [ ]:
#### ---- Crime Data Part 2 ---- ####

#Insert neighborhood ID into Crime Data
minne_crime_merge = minne_crime_transformed.merge(neighborhood_df[['neighborhoodid', 'neighborhood']], on=['neighborhood'])

#Remove neighborhood column
minne_crime_final = minne_crime_merge[["neighborhoodid","occurred_date","month", "year","monthyear","offense_cat","offense","latitude","longitude","crime_count"]]
minne_crime_final

### 2. Transform Demographic Data

#### 2a. Create dataset to hold the demographic categories to load to demo_cat table
* Concatonate the three dataframes(age, income, education) into one demographic dataframe
* Create demo_cat dataframe to hold only unique demographic (age, income, education) and category combinations

#### 2b Transform demographic dataframe to load to demographic_data table
* Use the demographic dataframe created in 2a
* Create demoid column and populate with the demoid value from the demo_cat dataframe on matches to demographic and category
* Remove demographic and category columns from the neighborhood demographic dataframe
* Rename the column headers

In [ ]:
#### ---- Demographic Categories ---- ####

#Concatonate the three dataframes(age, income, education) into one demographic dataframe
neighborhood_demos = pd.concat([education_df, age_df], ignore_index = True)
neighborhood_demos = pd.concat([neighborhood_demos, income_df], ignore_index = True)

#Get the list of unique demographic categories
demo_cat = neighborhood_demos.groupby(['demographic', 'Category']).size().reset_index().drop(columns=0)
demo_cat['demoid'] = demo_cat.index + 1


demo_cat

In [ ]:
#### ---- Neighborhood Demographic Data ---- ####

#Insert demoid
neighborhood_demos_merge = neighborhood_demos.merge(demo_cat[['demoid', 'Category', 'demographic']], on=['Category', 'demographic'])

#Insert neighborhoodid
neighborhood_demos_merge = neighborhood_demos_merge.merge(neighborhood_df[['neighborhoodid', 'neighborhood']], on=['neighborhood'])

#Delete demographic and category columns
neighborhood_demos_transformed = neighborhood_demos_merge[['neighborhoodid', 'demoid', 'Percentage']]

#Rename the column headers
neighborhood_demos_transformed = neighborhood_demos_transformed.rename(columns={"Percentage": "percent"})

#Rename demo_cat column headers
demo_cat = demo_cat.rename(columns={"Category": "category"})

neighborhood_demos_transformed

## Load 
##### Load the DataFrames containing neighborhood data, crime data and demographic data into the postgres database using pandas 'to_sql' command.

##### *Please note:  You may need to change the username, password and port data in the cell below in order to connect to your PostgreSQL database.

In [ ]:
##### Create Database Connection #####

protocol = 'postgresql'
username = 'postgres'
password = 'bootcamp'
host = 'localhost'
port = 5432
database_name = 'minne_crime_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
#Upload Neighborhoods
neighborhood_df.to_sql(name='neighborhooddata', con=engine, if_exists='append', index=False)

In [ ]:
#Upload Crime Data
minne_crime_final.to_sql(name='crimedata', con=engine, if_exists='append', index=False)

In [ ]:
#Upload Demographic Categories
demo_cat.to_sql(name='democat', con=engine, if_exists='append', index=False)

In [ ]:
#Upload Demogrpahics for neighborhoods
neighborhood_demos_transformed.to_sql(name='demographicdata', con=engine, if_exists='append', index=False)